# Convert JSON file to CSV and include review after 2017-12-31

In [12]:
#Load necessary libraries
import pandas as pd
import numpy as np
import json
import os
import seaborn as sns
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import codecs
from collections import defaultdict
import json
import dask.bag as db
import datetime
import dask.dataframe as dd

In [23]:
# We use default list to read yelp_academic_dataset_business.json and yelp_academic_dataset_user.json
d = defaultdict(list)

parent_dir = 'D:/DCU Documents/Semester 2/Data Analytics and Data Mining/Project/Kaggle Dataset/'

path_dic = {
            'B': parent_dir+'yelp_academic_dataset_business.json',
            'R':parent_dir+'yelp_academic_dataset_review.json',
            'U':parent_dir+'yelp_academic_dataset_user.json'
           }

#Used dask.bag to read yelp_academic_dataset_user.json beacuse of size of the file (5GB)
json_data = db.read_text(path_dic['R']).map(json.loads)
json_data.to_dataframe()

#Used pandas dataframe to read yelp_academic_dataset_user.json and yelp_academic_dataset_business.json
for key in path_dic:
    with codecs.open(path_dic[key],'r','utf-8') as f:
        d[key] = pd.DataFrame(json.loads(line) for line in f)

        

In [24]:
for k in d:
     d[k].to_csv('D:/DCU Documents/Semester 2/Data Analytics and Data Mining/Project/Kaggle Dataset/' + k + '.csv')

In [8]:
#Check the value of json_data
json_data.take(2)

({'review_id': 'lWC-xP3rd6obsecCYsGZRg',
  'user_id': 'ak0TdVmGKo4pwqdJSTLwWw',
  'business_id': 'buF9druCkbuXLX526sGELQ',
  'stars': 4.0,
  'useful': 3,
  'funny': 1,
  'cool': 1,
  'text': "Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four cheese filled pas

In [14]:
# Filter review yelp_academic_dataset_review.json to read only reviews after '2017-12-31'
d1 = datetime.datetime(2017, 12, 31)
json_data = json_data.filter(lambda record: datetime.datetime.strptime(record['date'], '%Y-%m-%d %H:%M:%S') > d1)

In [16]:
#Convert dask.bag to dask.dataframe

def flatten(record):
    return {
        'review_id': record['review_id'],
        'user_id': record['user_id'],
        'business_id': record['business_id'],
        'stars': record['stars'],
        'useful': record['useful'],
        'funny': record['funny'],
        'cool': record['cool'],
        'text': record['text'],
        'date' : record['date']
    }

df = json_data.map(flatten).to_dataframe()
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,J4a2TuhDasjn2k3wWtHZnQ,RNm_RWkcd02Li2mKPRe7Eg,xGXzsc-hzam-VArK6eTvtw,1.0,2,0,0,"This place used to be a cool, chill place. Now...",2018-01-21 04:41:03
1,ucFOnqgaV40oQ2YNyz5ddQ,JHXQEayrDHOWGexs0dCviA,KXCXaF5qimmtKKqnPc_LQA,1.0,0,0,0,Great coffee and pastries. Baristas are excell...,2018-03-03 23:45:25
2,vwPVD6AfV3Hi5F2bjKZK1w,g-1DDTwuJArYg2ZHolRAOw,vRrDTIW9IFBO4cc3laazUw,5.0,0,0,0,So sad to learn they are demolishing this. Alw...,2018-03-15 20:34:25
3,P320Yt8vFD3yjI34hq-umg,IMfkbLVt_GJfD7zJ9T4fcw,VPqWLp9kMiZEbctCebIZUA,3.0,0,0,0,"Overall is good, is worth the line up I don't ...",2018-09-25 03:22:50
4,EO5rALvJMkK8QEvUNs1gxg,u2xPfv6_wcKt-lW-C1cV8A,9P-lp3AWDXGayDqJz9VPwQ,2.0,0,0,0,The ramen here is less than great. It came out...,2018-02-11 03:30:12


In [15]:
#Checks if the filter of dates from 2018-12-31 worked
json_data.take(5)

({'review_id': 'J4a2TuhDasjn2k3wWtHZnQ',
  'user_id': 'RNm_RWkcd02Li2mKPRe7Eg',
  'business_id': 'xGXzsc-hzam-VArK6eTvtw',
  'stars': 1.0,
  'useful': 2,
  'funny': 0,
  'cool': 0,
  'text': 'This place used to be a cool, chill place. Now its a bunch of neanderthal bouncers hopped up on steroids acting like the can do whatever they want. There are so many better places in davis square where they are glad you are visiting their business. Sad that the burren is now the worst place in davis.',
  'date': '2018-01-21 04:41:03'},
 {'review_id': 'ucFOnqgaV40oQ2YNyz5ddQ',
  'user_id': 'JHXQEayrDHOWGexs0dCviA',
  'business_id': 'KXCXaF5qimmtKKqnPc_LQA',
  'stars': 1.0,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'text': 'Great coffee and pastries. Baristas are excellent. All the other staff are sooooo lost and vacant.',
  'date': '2018-03-03 23:45:25'},
 {'review_id': 'vwPVD6AfV3Hi5F2bjKZK1w',
  'user_id': 'g-1DDTwuJArYg2ZHolRAOw',
  'business_id': 'vRrDTIW9IFBO4cc3laazUw',
  'stars': 5.0,
  'u

In [17]:
#Convert to 'R.csv' from json file
df.to_csv('D:/DCU Documents/Semester 2/Data Analytics and Data Mining/Project/Kaggle Dataset/' + 'R1' + '.csv')

['D:/DCU Documents/Semester 2/Data Analytics and Data Mining/Project/Kaggle Dataset/R1.csv\\0.part']